In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [62]:
## Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

In [63]:
datapath = '../WeatherPy/citydata.csv'
city_weather_df = pd.read_csv(datapath)
city_weather_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,avera,33.19,-82.53,80.60,94,1,2.93,US,1599787500
1,dikson,73.51,80.55,44.51,99,100,20.47,RU,1599787501
2,saint-philippe,-21.36,55.77,66.20,77,40,6.93,RE,1599787501
3,port alfred,-33.59,26.89,63.28,86,96,19.37,ZA,1599787501
4,east london,-33.02,27.91,64.67,78,81,13.58,ZA,1599787502
...,...,...,...,...,...,...,...,...,...
564,pangkalanbuun,-2.68,111.62,83.19,72,99,3.69,ID,1599787663
565,tempio pausania,40.90,9.10,68.00,100,100,6.93,IT,1599787664
566,plouzane,48.38,-4.62,51.80,100,11,4.70,FR,1599787664
567,san borja,-14.82,-66.85,81.10,39,1,2.01,BO,1599787664


In [203]:
##Humidity Heatmap
# Configure gmaps
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

In [245]:
# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_weather_df["Humidity"].astype(float)

In [246]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [250]:
nice_weather_df = city_weather_df[(city_weather_df.Max_Temp < 75)
                                  & (city_weather_df.Max_Temp >= 55)
                                  & (city_weather_df.Cloudiness > 25)
                                  & (city_weather_df.Cloudiness < 60)
                                  & (city_weather_df.Wind_Speed < 11)
                                  & (city_weather_df.Humidity > 25)
                                  & (city_weather_df.Humidity < 75)]
#nice_weather_df

In [251]:
hotels_df = nice_weather_df[['City','Lat','Lng','Country']].copy()
#hotels_df

In [252]:
#Add Columns for Hotel Name, City and Country
hotels_df['Name'] = ""
hotels_df['Rating'] = ""
hotels_df

,City,Lat,Lng,Country,Name,Rating
66,yakeshi,49.28,120.73,CN,,
115,acari,-15.43,-74.62,PE,,
129,ubatuba,-23.43,-45.07,BR,,
283,manono,-7.30,27.42,CD,,
285,port lincoln,-34.73,135.87,AU,,
347,nikolayevsk-na-amure,53.14,140.73,RU,,
368,zeya,53.75,127.27,RU,,
375,ulladulla,-35.35,150.47,AU,,
517,belaya gora,68.53,146.42,RU,,


In [253]:
# geocoordinates
# target_coordinates = "43.6187102, -116.2146068"
target_search = "Resort"
target_radius = 5000
target_type = "lodging"
# set up a parameters dictionary
params = {
#    "location": target_coordinates,
#    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [254]:
# find the closest hotel of each type to coordinates

# use iterrows to iterate through pandas dataframe
for index, row in hotels_df.iterrows():

    # get City from df
    city = row['City']

    # add keyword to params dict
    params['location'] = f"{row['Lat']},{row['Lng']}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotels_df.loc[index, 'Name'] = results[0]['name']
        hotels_df.loc[index, 'Rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 66: yakeshi.
Closest hotel is Longhua Hotel.
Missing field/result... skipping.
------------
Retrieving Results for Index 115: acari.
Closest hotel is Hospedaje Ximens.
Missing field/result... skipping.
------------
Retrieving Results for Index 129: ubatuba.
Closest hotel is Hotel Pousada Dellamares.
------------
Retrieving Results for Index 283: manono.
Closest hotel is Hotel hewa bora.
Missing field/result... skipping.
------------
Retrieving Results for Index 285: port lincoln.
Closest hotel is Port Lincoln Hotel.
------------
Retrieving Results for Index 347: nikolayevsk-na-amure.
Closest hotel is Mini-hotel "Quiet Harbor".
------------
Retrieving Results for Index 368: zeya.
Closest hotel is Kapital Inn.
------------
Retrieving Results for Index 375: ulladulla.
Closest hotel is Sandpiper Motel.
------------
Retrieving Results for Index 517: belaya gora.
Missing field/result... skipping.
------------


In [255]:
hotels_df

,City,Lat,Lng,Country,Name,Rating
66,yakeshi,49.28,120.73,CN,Longhua Hotel,
115,acari,-15.43,-74.62,PE,Hospedaje Ximens,
129,ubatuba,-23.43,-45.07,BR,Hotel Pousada Dellamares,4.2
283,manono,-7.30,27.42,CD,Hotel hewa bora,
285,port lincoln,-34.73,135.87,AU,Port Lincoln Hotel,4.3
347,nikolayevsk-na-amure,53.14,140.73,RU,"Mini-hotel ""Quiet Harbor""",3.8
368,zeya,53.75,127.27,RU,Kapital Inn,4.3
375,ulladulla,-35.35,150.47,AU,Sandpiper Motel,4.2
517,belaya gora,68.53,146.42,RU,,


In [256]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df[['Name','City','Country','Rating']].iterrows()]
hotel_locations = hotels_df[["Lat", "Lng"]]
hotel_locations

,Lat,Lng
66,49.28,120.73
115,-15.43,-74.62
129,-23.43,-45.07
283,-7.30,27.42
285,-34.73,135.87
347,53.14,140.73
368,53.75,127.27
375,-35.35,150.47
517,68.53,146.42


In [258]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# markers = gmaps.marker_layer(hotel_locations)
markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))